In [7]:
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from time import time as timer

In [2]:
beta = 2/7 # 3/7, 1.5/10
gamma = 1/7 # 1/7, 1/10 
Reff = beta/gamma # Effective reproduction number


omega = beta + gamma # Average lifetime of infectious case
r = beta - gamma # Growth rate 
rho = omega


ntime = 10000
Tmax = 1000
time = np.linspace(0, Tmax, ntime)
dtime = np.diff(time)[0]

# Initial conditions for probability of extinction and moment equations

q0 = 0
m0 = 1
m2_0 = 1

# Colour map

cmap = sns.color_palette("Set2")


In [3]:
stop_times_df2 = pd.read_csv("stop_times_df2.csv")

In [9]:
def gillespie_peak_time(beta, gamma, y0, Tmax = Tmax, N = int(1e7)):
    nevents = int(2*N)
    omega = beta + gamma
    t = 0
    rands = np.random.random(2*nevents).reshape((nevents, 2))
    cases = np.nan * np.ones((nevents, 2))
    cases[0, 1] = y0
    cases[0, 0] = N-y0
    time = [0]
    e=0
    while cases[e, 1]>0:
        e+=1
        rate_vec = np.array((beta*cases[e-1, 1]*cases[e-1, 0]/N, gamma*cases[e-1, 1]))
        rate_sum = np.sum(rate_vec)

        rate_cumsum = np.cumsum(rate_vec)
        dt = -np.log(rands[e, 0])/rate_sum
        t += dt
        
        time.append(t)
        event_rand = rate_sum * rands[e, 1]
        try:
            event = np.where(rate_cumsum >= event_rand)[0][0]
        except:
            print(e, cases[0, :], rate_sum)
            return cases
        if event ==0:
            cases[e, 1] = cases[e-1, 1] + 1
            cases[e, 0] = cases[e-1, 0] - 1
        elif event == 1:
            cases[e, 1] = cases[e-1, 1] - 1
            cases[e, 0] = cases[e-1, 0]
        
    cases[e:, 1] = 0
    peak_size = np.max(cases[:, 1])
    peak_idx = np.argmax(cases[:, 1])
    peak_time = time[peak_idx]
    if peak_size >= N/1000:
        return peak_time, peak_size
    else:
        return 0., 0.

In [15]:
np.random.seed(123)
nruns_peak = 1000
population = int(1e7)

start = timer()
peaks = Parallel(n_jobs=-1)(delayed(gillespie_peak_time)(beta, 
                                                         gamma, 
                                                         y0 = 1,
                                                         N = population
                                                        ) for i in range(nruns_peak))
print("Executed in " + str(timer()-start) + ' seconds')
peak_times = pd.DataFrame(np.array([pt for (pt, ps) in peaks]))
peak_times.columns = ["PeakTime"]
print(peak_times)
peak_times.to_csv("peak_times_homogeneity_R=" + str(Reff) + "_nruns=" + str(nruns_peak) + "_pop=" + str(population) + ".csv")


/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Executed in 6319.022735118866 seconds
       PeakTime
0      0.000000
1    110.429978
2    109.300377
3    107.374390
4    112.225171
..          ...
995    0.000000
996    0.000000
997  105.203782
998  106.805526
999  103.050763

[1000 rows x 1 columns]


In [ ]:
plt.figure(7)
peak_df = pd.DataFrame(peak_times[peak_times>0], columns = ["Peak (full simulation)"])
sns.kdeplot(stop_times_df2[cols[st_idx]]+ peak_time, label = "Gillespie Simulated FPT", color = 'purple', alpha = 0.5)
sns.kdeplot(peak_df["Peak (full simulation)"], label = "Peak (full simulation)")

plt.xlim([75, 200])
plt.xlabel("Time (Days)")
plt.title("Peak time distribution, R = 2")
plt.legend()
plt.figure(8)


plt.plot(time[:peak_idx], np.zeros(peak_idx), color = 'purple')

ecdf_peak = np.zeros_like(time)
ecdf_peak_approx = np.zeros_like(time)
st_df = stop_times_df2[cols[st_idx]].dropna()
for t_idx in tqdm(range(len(time))):
    T = time[t_idx]
    ecdf_peak[t_idx] = len(np.where(peak_df <= T)[0])/len(peak_df)
    ecdf_peak_approx[t_idx] = len(np.where( st_df <= T)[0])/len(st_df)

plt.plot(time, ecdf_peak, label = "FPT (Full simulation)")
plt.plot(time + peak_time, ecdf_peak_approx, label = "Gillespie Simulated FPT", color = 'tab:purple', alpha = 0.5)
plt.xlim([0, 200])
plt.ylim([0, 1])
plt.xlabel("Time (Days)")
plt.title("Peak time distribution, R = 2")
plt.legend()
plt.savefig("True_peak_time_R=" + str(Reff) + ".pdf")